in this notebook we:
* train stage 1 and stage 2 for longer time (6 epochs not 3)

In [1]:
from pathlib import Path
import json
import logging

import numpy as np
import pandas as pd

from fastai.vision import *

import os

import wandb
from wandb.fastai import WandbCallback

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import configparser

from functions_wandb import *

In [2]:
# ! pip install wandb

In [3]:
# logging.basicConfig(level=logging.INFO)

In [4]:
wandb.init(project="gabon")

W&B Run: https://app.wandb.ai/jedrzej/gabon/runs/1ib0zu0a

In [5]:
# PATH_TO_IMG = Path("/data_rescaled")

# PATH_TO_MAIN = Path("/home/jupyter/")
# PATH_TO_TRAIN_DF = PATH_TO_MAIN / "inspect_data_split_validation"
# PATH_TO_MODELS = PATH_TO_MAIN / "training" / "saved_models"
# PATH_TO_CONFIG = PATH_TO_MAIN / "config"

# CONFIG = configparser.ConfigParser()
# CONFIG.read(PATH_TO_CONFIG / "neptune.ini")

In [6]:
# ! pip install neptune-client neptune-contrib

In [7]:
# neptune.init(project_qualified_name='appsilon/gabon-wildlife', 
#             api_token=CONFIG["neptune"]["NEPTUNE_API_TOKEN"])

In [8]:
df = pd.read_csv(PATH_TO_TRAIN_DF / "train_valid_df.csv")

classes = df.species.unique()
print(len(classes)) # no "Mammal_Other"
classes

28


array(['Human', 'Blank', 'Elephant_African', 'Hog_Red_River', 'Buffalo_African', 'Leopard_African', 'Monkey',
       'Duiker_Red', 'Civet_African_Palm', 'Squirrel', 'Duiker_Blue', 'Bird', 'Mongoose_Black_Footed', 'Rodent',
       'Duiker_Yellow_Backed', 'Genet', 'Chimpanzee', 'Gorilla', 'Mongoose', 'Porcupine_Brush_Tailed', 'Pangolin',
       'Mandrillus', 'Chevrotain_Water', 'Cat_Golden', 'Rat_Giant', 'Guineafowl_Crested', 'Guineafowl_Black',
       'Rail_Nkulengu'], dtype=object)

In [9]:
len(df.uniqueName)

347120

In [10]:
df.head()

,uniqueName,species,main_folder,is_valid,sub_folder
0,0000005.jpg,Human,Compressed Camera Trap Images,False,T33
1,0000007.jpg,Human,Compressed Camera Trap Images,False,T33
2,0000009.jpg,Human,Compressed Camera Trap Images,False,T33
3,0000013.jpg,Human,Compressed Camera Trap Images,False,T33
4,0000015.jpg,Human,Compressed Camera Trap Images,False,T33


In [11]:
df["uniqueName"] = df.uniqueName.apply(lambda x: "resc_" + x)
df.head()

,uniqueName,species,main_folder,is_valid,sub_folder
0,resc_0000005.jpg,Human,Compressed Camera Trap Images,False,T33
1,resc_0000007.jpg,Human,Compressed Camera Trap Images,False,T33
2,resc_0000009.jpg,Human,Compressed Camera Trap Images,False,T33
3,resc_0000013.jpg,Human,Compressed Camera Trap Images,False,T33
4,resc_0000015.jpg,Human,Compressed Camera Trap Images,False,T33


In [12]:
%%time
data = get_training_data(df, (384 // 4, 512 // 4))#, partial_pct=0.01)

CPU times: user 9.83 s, sys: 328 ms, total: 10.2 s
Wall time: 5.54 s


In [13]:
# %%time
# data.show_batch()

# Stage 1

In [14]:
learn = get_initial_learner(data)

In [15]:
# %%time
# run_find_lr(learn, "stage1-1epochs-96_128-rescaled")

In [16]:
initial_lr = 1e-3

In [17]:
# %%time
# # with neptune.create_experiment(params = {"initial_lr": initial_lr}): 
# #     learn.callbacks.append(NeptuneMonitor())
# run_training(learn, "stage1-1epochs-96_128-rescaled", lr=initial_lr, n_epochs=1)

In [18]:
# learn = load_learner(PATH_TO_MODELS, "stage1-1epochs-96_128-rescaled.pkl")
# learn.data = data
learn = get_initial_learner(data)
learn.load(PATH_TO_MODELS / "stage1-1epochs-96_128-rescaled");

# Stage 2

In [19]:
learn.unfreeze()
# run_find_lr(learn, "stage2-3epochs-96_128-rescaled")

In [20]:
initial_lr = 1e-3
lr = 1e-5
lr_end = initial_lr / 5
n_epochs =  6

In [21]:
# with neptune.create_experiment(params = {
#                                         "initial_lr": initial_lr,
#                                         "lr": lr,
#                                         "lr_end": lr_end,
#                                         "n_epochs": n_epochs
#                                         }): 
#     learn.callbacks.append(NeptuneMonitor())
run_training(learn=learn, model_name="stage2-6epochs-96_128-rescaled", lr=lr, lr_end=lr_end, n_epochs=n_epochs)

epoch,train_loss,valid_loss,accuracy,time
0,0.962095,1.842613,0.500733,22:45
1,0.655957,1.862645,0.508733,22:58
2,0.495792,1.797578,0.555286,22:50
3,0.393088,1.777252,0.555309,22:26
4,0.339182,1.882536,0.550954,22:29
5,0.317027,1.888605,0.554254,22:23


Better model found at epoch 0 with valid_loss value: 1.8426131010055542.
Better model found at epoch 2 with valid_loss value: 1.7975778579711914.
Better model found at epoch 3 with valid_loss value: 1.7772517204284668.
Loaded best saved model from /home/jupyter/training/wandb/run-20200311_214730-1ib0zu0a/bestmodel.pth


In [22]:
learn.recorder.plot_losses()

In [23]:
interp = ClassificationInterpretation.from_learner(learn)

In [24]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)